In [3037]:
import requests
import csv
import numpy as np
import pandas as pd
from unidecode import unidecode
import time
from bs4 import BeautifulSoup

In [4901]:
fights_url = "http://ufcstats.com/statistics/events/completed?page=all"

data = requests.get(fights_url) # This gets the entire webpages HTML

In [4902]:
soup = BeautifulSoup(data.text) # gives us a BeautifulSoup object, which represents
                                # the document as a nested data structure. Now we can use the BeautifulSoup library
                                # to pull data out of the HTML (parser)

In [4903]:
fight_table = soup.select("table.b-statistics__table-events")[0] 
# Selects all of the HTML within <table class="b-statistics__table-events"</table>

In [4904]:
links = fight_table.find_all('a')

links = [l.get("href") for l in links]

fight_urls = links

# This section: requests each fight-card url individually, reads it, and adds the card to a df. It uses BeautifulSoup to parse the datetime. The date is split and striped from the time, and the string is converted to a timestamp. the date is added to the card df. The fighters names are seperated into winner and loser. The card df is concatinated onto itself. The time between loops is set to .5 seconds to avoid being disconnected by the website.

In [4905]:
df = pd.DataFrame() # data frame being created
for fight_url in fight_urls:
    data2 = requests.get(fight_url) # requesting each fight card individually
    fights = pd.read_html(data2.text)
    card = fights[0] # indexing to get each individual card
    soup = BeautifulSoup(data2.text)
    date = soup.select("li.b-list__box-list-item") # uses bs to find date, separate date, and convert
    date1 = str(date[0])                           # date to date time.
    date2 = date1.split('\n')
    date = date2[-2].strip()
    date = pd.to_datetime(date)
    card['date'] = date
    card[['w_fighter','l_fighter']] = card['Fighter'].str.split('  ', expand=True) # Both fighters are in the same
    # cell. This sperates them into winner and loser.
    df = pd.concat([df, card])
    time.sleep(.5)
    #count += 1


In [4910]:
#droping unneeded columns

df = df.drop(['Fighter', 'Kd', 'Str', 'Td', 'Sub', 'Weight class', 'Method', 'Round', 'Time'], axis=1)

In [4911]:
# saving work

df.to_csv('df', index=False)

# Requests all the links from the fighters section of the website. The fighters are seperated alphabetically, so this puts each letter's url into a list. It also adds the beginning of the url for each fighter.

In [3782]:
fighters_url = 'http://ufcstats.com/statistics/fighters?char=s&page=all'

data3 = requests.get(fighters_url)

soup2 = BeautifulSoup(data3.text)

alpha_links = soup2.select('ul.b-statistics__nav-items')[0]

fighter_table = alpha_links.find_all('a')

table_links = [l.get("href") for l in fighter_table]

tl2 = table_links

tl2 = list(set(tl2))

alpha_urls = [f"http://ufcstats.com{l}" for l in tl2]

# Using the urls in alpha_urls, creating a df for the fighters. This includes height and reach. 

In [ ]:
fighter_df = pd.DataFrame()
for letter in alpha_urls:
    data5 = requests.get(letter)
    soup3 = BeautifulSoup(data5.text)
    full_letter = soup3.select('li.b-statistics__paginate-item')[-1]
    full_table = full_letter.find_all('a')
    ft_link = [l.get('href') for l in full_table]
    data6 = requests.get(ft_link[0])
    fighters = pd.read_html(data6.text)
    page = fighters[0]
    fighter_df = pd.concat([fighter_df, page])
    time.sleep(.5)


# Drop unneeded columns. Combine the columns 'First' and 'Last' into column 'fighter'.

In [4376]:
fighter_df = fighter_df.drop(['Nickname', 'Stance', 'W', 'L', 'D', 'Belt'], axis=1)

fighter_df['fighter'] = fighter_df['First'] + ' ' + fighter_df['Last']

fighter_df = fighter_df.reset_index()

fighter_df = fighter_df.drop(['index'], axis=1)

fighter_df = fighter_df.drop(['First', 'Last'], axis=1)

In [3056]:
fighter_df.to_csv('fighter_df', index=False)

# df2 reverses fighter and opponent and gives all the fighters losses. Then it will be concatenated with df and both fighters from every fight will have their own row. df3 is the resulting table.

In [4924]:
# creating a new df to alter

df2 = df.copy()

In [5276]:
# changing the result from all wins to all losses

df2['W/L'][df2['W/L'] == 'win'] = 'loss'  

In [4925]:
# assigns the losing fighter to 'fighter' and the winning fighter to 'opponent'

df2 = df2.rename(columns={'fighter':'opponent', 'opponent':'fighter'})

In [4907]:
# assingns the winning fighter to 'fighter' and the losing fighter to 'opponent'

df = df.rename(columns={'w_fighter':'fighter', 'l_fighter':'opponent'})

In [4928]:
# doubles the rows. Each fighter is now in the 'figther' column and the 'opponent' column for each fight.

df3 = pd.concat([df.copy(),df2.copy()])

In [5278]:
df3

,win,date,fighter,opponent,fighter_ht,fighter_wt,fighter_reach,opponent_ht,opponent_wt,opponent_reach
0,NaN,2023-11-04,Jailton Almeida,Derrick Lewis,"6' 3""",205 lbs.,"79.0""","6' 3""",260 lbs.,"79.0"""
1,0,2018-10-06,Alexander Volkov,Derrick Lewis,"6' 7""",250 lbs.,"80.0""","6' 3""",260 lbs.,"79.0"""
2,1,2021-08-07,Ciryl Gane,Derrick Lewis,"6' 4""",245 lbs.,"81.0""","6' 3""",260 lbs.,"79.0"""
3,0,2023-07-29,Marcos Rogerio de Lima,Derrick Lewis,"6' 1""",253 lbs.,"75.0""","6' 3""",260 lbs.,"79.0"""
4,1,2022-07-30,Sergei Pavlovich,Derrick Lewis,"6' 3""",257 lbs.,"84.0""","6' 3""",260 lbs.,"79.0"""
...,...,...,...,...,...,...,...,...,...,...
14785,0,1997-02-07,Eric Martin,Justin Martin,--,--,--,"6' 0""",185 lbs.,--
14786,0,1996-05-17,Matt Andersen,Rafael Carino,"6' 2""",230 lbs.,--,"6' 8""",245 lbs.,--
14787,0,1995-09-08,Francesco Maturi,Onassis Parungao,--,--,--,"5' 11""",200 lbs.,--
14788,0,1995-04-07,Andy Anderson,Jon Hess,"5' 6""",240 lbs.,--,"6' 7""",295 lbs.,--


In [4930]:
# changes the result columns name to 'win'

df3 = df3.rename(columns={'W/L':'win'})

# df3 is merged with fighter_df to add the height, weight, and reach of each fighter.

In [4931]:
df3 = df3.merge(fighter_df, left_on='fighter', right_on='fighter')

# The win column is converted to numerals. This will be the label when we create a ML model.

In [4934]:
df3['win'][df3['win']=='win']=1
df3['win'][df3['win']=='loss']=0
df3['win'][df3['win']== 'draw  draw']=2
df3['win'][df3['win']== 'nc  nc']=2

In [4935]:
# Renaming columns for clarity

df3 = df3.rename(columns={'Ht.':'fighter_ht', 'Wt.':'fighter_wt', 'Reach':'fighter_reach'})

# Merging each fighter in fighter_df onto the opponent in df3. This adds height, weight, and reach for each opponent.

In [4936]:
df3 = df3.merge(fighter_df, left_on='opponent', right_on='fighter')

In [3541]:
df3.to_csv('df3', index=False)

In [4940]:
# renaming columns for clarity

df3 = df3.rename(columns={'Ht.':'opponent_ht', 'Wt.':'opponent_wt', 'Reach':'opponent_reach', 'fighter_x':'fighter'})\
.drop(['fighter_y'], axis=1)

# This takes the name of every fighter from our list of fights and creates a wikipedia link. These links give us access to a table of fights for each fighter, giving us the records of the fighters. We can use this to establish the record of both fighters before the 'win' result happens.

In [ ]:
fighter_names = df3['fighter'].tolist()

lst2 = []
for _ in fighter_names:
    _ = _.replace(' ','_')
    lst2.append(_)
    
wiki_url = [f"https://en.wikipedia.org/wiki/{l}" for l in lst2]

In [3093]:
len(wiki_url)

14622

In [ ]:
m3 = pd.DataFrame()
for i in [2,3,4,5,-1,-2,-3,-4,-5]:
    for fighter in wiki_url:
        data7 = requests.get(fighter)
        y = pd.read_html(data7.text)
        try:
        soup7 = BeautifulSoup(data7.text)
        table = y[i] 
        head = soup7.head
        name = head.title.string
        fighter = name.split('-')[0].strip()
        table['fighter'] = unidecode(fighter)
        m3 = pd.concat([m3, table])
        except:
            continue
    time.sleep(.5)


# Creating a new df to alter

In [ ]:
df10 = df3.copy()

In [4592]:
df10.to_csv('df10', index=False)

# Changing the names of the columns, reducing the number of columns, removing NaNs, changing date type to datetime.

In [3699]:
m3 = m3.rename(columns={'Date':'date', 'Opponent':'opponent', 'Record':'record'})

m3 = m3[['date', 'fighter', 'opponent', 'record']]

m3 = m3[~m3['record'].isna()]

m3['date'] = pd.to_datetime(m3['date'], errors='coerce')

m3 = m3[~m3['date'].isna()]

#m3['fighter'] = m3['fighter'].map(lambda x: x.rstrip('(fighter)'))

# Merging m3 to df10. merging the two record columns into one. This is usful if I have to create multiple m3s to scrape every wiki table. I hope to make m3 complete on the first scrape.

In [3700]:
df10 = df10.merge(m3, how='left', on=['date', 'fighter'])

df10 = df10.rename(columns={'opponent_x': 'opponent'})

df10 = df10.drop(['opponent_y'], axis=1)

df10['record'] = np.where(df10['record_x'].isnull(), df10['record_y'], df10['record_x'])

df10 = df10.drop(['record_x', 'record_y'], axis=1)

In [4468]:
# There are two Bruno Silvas. This seperates out the Bruno Silva that will stay in the df

j = df10[((df10['fighter'] == 'Bruno Silva')&(df10['fighter_wt']!='185 lbs.'))\
         |(df10['opponent'] == 'Bruno Silva')&(df10['opponent_wt']!='185 lbs.')|\
        (df10['opponent'] == 'Bruno Silva')& (df10['fighter_wt'] == '125 lbs.')|\
        (df10['opponent'] == 'Bruno Silva')& (df10['fighter_wt'] == '135 lbs.')].index
df10.drop(j, inplace=True)

In [4455]:
# when I read in the saved df, I have to convert the date to datetime.

df10 = pd.read_csv('df10')

df10['date'] = pd.to_datetime(df10['date'], errors='coerce')

In [5279]:
df10

,win,date,fighter,opponent,fighter_ht,fighter_wt,fighter_reach,opponent_ht,opponent_wt,opponent_reach,record
0,0,2023-09-16,Alexa Grasso,Valentina Shevchenko,"5' 5""",125 lbs.,"66.0""","5' 5""",125 lbs.,"66.0""",16–3–1
1,1,2023-03-04,Alexa Grasso,Valentina Shevchenko,"5' 5""",125 lbs.,"66.0""","5' 5""",125 lbs.,"66.0""",16–3
2,1,2017-09-09,Amanda Nunes,Valentina Shevchenko,"5' 8""",135 lbs.,"69.0""","5' 5""",125 lbs.,"66.0""",15–4
3,1,2016-03-05,Amanda Nunes,Valentina Shevchenko,"5' 8""",135 lbs.,"69.0""","5' 5""",125 lbs.,"66.0""",12–4
4,0,2016-07-23,Holly Holm,Valentina Shevchenko,"5' 8""",135 lbs.,"69.0""","5' 5""",125 lbs.,"66.0""",10–2
...,...,...,...,...,...,...,...,...,...,...,...
13781,0,2017-12-01,Melinda Fabian,DeAnna Bennett,"5' 6""",125 lbs.,"66.0""","5' 4""",125 lbs.,"68.0""",4–3–2
13782,0,2017-09-02,Bojan Mihajlovic,Abdul-Kerim Edilov,"5' 11""",205 lbs.,"73.0""","6' 2""",205 lbs.,--,NaN
13783,0,2014-07-06,Keith Berish,Robert Drysdale,"6' 0""",205 lbs.,--,"6' 3""",205 lbs.,--,NaN
13784,0,2006-12-13,Steve Byrnes,Logan Clark,"6' 0""",170 lbs.,--,"6' 2""",185 lbs.,--,NaN


In [3707]:
# renaming and dropping columns from the merged df

df10 = df10.rename(columns={'fighter_ht_x'\
                         :'fighter_ht', 'fighter_wt_x':'fighter_wt', 'fighter_reach_x':'fighter_reach'})

df10 = df10.drop(['fighter_ht_y','fighter_wt_y', 'fighter_reach_y'],axis=1)

# Merging df10 to itself, joining on 'fighter' == 'opponent' and 'date' == 'date'. gives me the opponents record on the event date.

In [4962]:
df11 = df10.copy().merge(df10.copy(), left_on=['opponent', 'date'], right_on=['fighter', 'date'])

In [4963]:
# renaming columns after merge.

df11 = df11.rename(columns={'win_x': 'win', 'fighter_x':'fighter', 'opponent_x':'opponent', 'fighter_ht_x'\
                           :'fighter_ht', 'fighter_wt_x':'fighter_wt', 'fighter_reach_x':'fighter_reach'\
                           ,'opponent_ht_x':'opponent_ht', 'opponent_wt_x':'opponent_wt', 'opponent_reach_x':\
                           'opponent_reach', 'record_x':'record', 'record_y':'opponent_record'})

In [4964]:
# keeping the columns I want

df11 = df11[['win', 'date', 'fighter', 'opponent', 'fighter_ht', 'fighter_wt', 'fighter_reach', 'opponent_ht',\
            'opponent_wt', 'opponent_reach', 'record', 'opponent_record']]

# Spiltting strings into ints to be used as features

In [4971]:
# splits the heights (inches from feet), converts feet to inches, returns height in inches
df11['opponent_ht'] = df11['opponent_ht'].map(lambda x: x.replace('\'', '').strip())
df11['opponent_ht'] = df11['opponent_ht'].map(lambda x: x.replace('\"', '').strip())

df11['fighter_ht'] = df11['fighter_ht'].map(lambda x: x.replace('\'', '').strip())
df11['fighter_ht'] = df11['fighter_ht'].map(lambda x: x.replace('\"', '').strip())

df11[['feet', 'inches']] = df11['opponent_ht'].str.split(' ', expand=True)

df11[['ffeet', 'finches']] = df11['fighter_ht'].str.split(' ', expand=True)

df11['feet'] = pd.to_numeric(df11['feet'], errors='coerce').astype('Int64')

df11['ffeet'] = pd.to_numeric(df11['ffeet'], errors='coerce').astype('Int64')

df11['inches'] = pd.to_numeric(df11['inches'], errors='coerce').astype('Int64')

df11['finches'] = pd.to_numeric(df11['finches'], errors='coerce').astype('Int64')

df11['feet'] = df11['feet']*12

df11['ffeet'] = df11['ffeet']*12

df11['opponent_ht'] = df11['feet']+df11['inches']

df11['fighter_ht'] = df11['ffeet']+df11['finches']

df11 = df11.drop(['feet', 'inches', 'ffeet', 'finches'], axis=1)

# weight is unneeded because the weights are the same for each fighter in the fight.

df11 =df11.drop(['fighter_wt','opponent_wt'], axis=1)

# doing the same for reach as was done for height.

df11['fighter_reach'] = df11['fighter_reach'].map(lambda x: x.replace('\"', '').strip())
df11['opponent_reach'] = df11['opponent_reach'].map(lambda x: x.replace('\"', '').strip())


df11['fighter_reach'] = df11['fighter_reach'].map(lambda x: x.replace('.0', '').strip())
df11['opponent_reach'] = df11['opponent_reach'].map(lambda x: x.replace('.0', '').strip())

df11['fighter_reach'] = pd.to_numeric(df11['fighter_reach'], errors='coerce').astype('Int64')
df11['opponent_reach'] = pd.to_numeric(df11['opponent_reach'], errors='coerce').astype('Int64')

# converting the records to str then splint wins, losses, and draws into individual columns before
# converting them to integers.

df11['record'] = df11['record'].str.split('(').str[0]
df11['opponent_record'] = df11['opponent_record'].str.split('(').str[0]

df11['record'] = df11['record'].str.replace(r'[\W]', ' ', regex=True)

df11['opponent_record'] = df11['opponent_record'].str.replace(r'[\W]', ' ', regex=True)

df11[['wins', 'losses']] = df11['record'].str.split(' ',1, expand=True)

df11[['losses', 'draw']] = df11['losses'].str.split(' ',1, expand=True)

df11[['opp_wins', 'opp_losses']] = df11['opponent_record'].str.split(' ',1, expand=True)

df11[['opp_losses', 'opp_draw']] = df11['opp_losses'].str.split(' ',1, expand=True)

df11['opp_wins'] = pd.to_numeric(df11['opp_wins'], errors='coerce').astype('Int64')
df11['opp_losses'] = pd.to_numeric(df11['opp_losses'], errors='coerce').astype('Int64')

df11['wins'] = pd.to_numeric(df11['wins'], errors='coerce').astype('Int64')
df11['losses'] = pd.to_numeric(df11['losses'], errors='coerce').astype('Int64')

# Many fighters do not have a draw on their record, so NA values need to be replaced with zeros
df11['opp_draw'] = df11['opp_draw'].fillna(0)

df11['draw'] = df11['draw'].fillna(0)

df11['draw'] = pd.to_numeric(df11['draw'], errors='coerce').astype('Int64')
df11['opp_draw'] = pd.to_numeric(df11['opp_draw'], errors='coerce').astype('Int64')

# df11 looks good. Now I will use a copy of it to make more changes

In [4973]:
df12 = df11.copy().drop(['record','opponent_record'], axis=1)

# The records on wikipedia are the records after the event. To get the records of the fighters before the event, wins and losses must be subtracted. The label must be predicted using information that was available before the fight took place.

In [4974]:
# This cell can only be ran once or it will continue to subtract and add resulting in errors in the data

df12.loc[df12['win']==0, ['opp_wins']] = df12['opp_wins'].subtract(1)

df12.loc[df12['win']==0, ['losses']] = df12['losses'].subtract(1)

df12.loc[df12['win']==1, ['wins']] = df12['wins'].subtract(1)

df12.loc[df12['win'] == 1, ['opp_losses']] = df12['opp_losses'].subtract(1)

In [4975]:
# looking for negative values in the loss column that have showed up because some fighters have zero
# losses.

df12.loc[df12['losses']<0, ['losses']] = 0

In [4978]:
# keeping what we need and creating a new df.

df12 = df12[['win', 'date', 'fighter', 'opponent', 'fighter_ht', 'fighter_reach', 'opponent_ht', \
            'opponent_reach', 'wins', 'losses', 'draw', 'opp_wins', 'opp_losses', 'opp_draw']]
df13 = df12.copy()

In [4980]:
# sorting by fighter and date

df14 = df13.copy().sort_values(['fighter', 'date'], ascending=False).reset_index()

# dropping index row created by reseting the index.

df14 = df14.drop(['index'], axis=1)

In [4981]:
# saving work

df14.to_csv('df14', index=False)

In [5216]:
df15 = df14.copy()

# This fills in the missing wins and losses for the fighters. This back fills NAs going forward in time.

In [5220]:
# This has to be ran multiple times.

df15['ww'] = df15['wins'][df15['win']==1].add(1)

df15['ww'][df15['win']==0] = df15['wins']
df15['ww'][df15['win']==2] = df15['wins']

df15['ww'] = df15.groupby('fighter')['ww'].fillna(method='bfill', limit=1)

df15['wins'][df15['wins'].isna()] = df15['ww']

df15['wl'] = df15['losses'][df15['win'] == 0].add(1)
df15['wl'][df15['win']==1] = df15['losses']

df15['wl'][df15['win']==2] = df15['losses']

df15['wl'] = df15.groupby('fighter')['wl'].fillna(method='bfill', limit=1)

df15['losses'][df15['losses'].isna()] = df15['wl']

# This fills in missing wins and losses going backwards (front fills) in time.

In [5284]:
#this has to be ran multiple times.
df15['ffw'] = df15['wins'][df15['win']==1].add(-1)

df15['ffw'][df15['win']==0] = df15['wins']
df15['ffw'][df15['win']==2] = df15['wins']

df15['ffw'] = df15.groupby('fighter')['ffw'].fillna(method='ffill', limit=1)

df15['wins'][df15['wins'].isna()] = df15['ffw']

df15['ffl'] = df15['losses'][df15['win'] == 0].add(-1)
df15['ffl'][df15['win']==1] = df15['losses']
df15['ffl'][df15['win']==2] = df15['losses']

df15['ffl'] = df15.groupby('fighter')['ffl'].fillna(method='ffill', limit=1)

df15['losses'][df15['losses'].isna()] = df15['ffl']

# Run the cells above and then check for missing 'win' column values. When this number doesn't go down anymore, the above cells don't need to be ran anymore.

In [5285]:
# Checks if our front-filling and back-filling are complete.
df15['wins'].isna().sum()

1888

In [5228]:
# dropping the columns created to backfill and frontfill wins and losses.

df15 = df15.drop(['ww','wl', 'ffl', 'ffw'], axis=1)

# Now that every fighters record is correct, we fill in the records when each combatant was the opponent.

In [5230]:
#filling in the missing opp wins and losses by merging
temp = df15[['date', 'fighter', 'wins', 'losses']]

In [5231]:
df15 = df15.merge(temp, left_on=['opponent','date'], right_on=['fighter','date'])

# Cleaning after merger

In [5233]:
df15 = df15.drop(columns={'opp_wins', 'opp_losses','fighter_y'})

In [5234]:
df15 = df15.rename(columns={'fighter_x':'fighter', 'wins_x':'wins', 'losses_x':'losses', 'wins':'opp_wins',\
                   'losses':'opp_losses', 'wins_y':'opp_wins', 'losses_y':'opp_losses'})

In [5236]:
df15.to_csv('df15', index=False)

In [5305]:
df15 = df15.drop_duplicates()

df15['opp_draw'] = df15['opp_draw'].fillna(0)

df15['draw'] = df15['draw'].fillna(0)

df_f = df15.dropna()

In [5307]:
df_f.to_csv('df_f', index=False)

# Getting access to the fighters individual pages to extract date of birth

In [4498]:
l = []
for letter in alpha_urls:
    try:
        data = requests.get(letter)
        soup = BeautifulSoup(data.text)
        full_letter = soup.select('li.b-statistics__paginate-item')[-1]
        full_table = full_letter.find_all('a')
        ft_link = [l.get('href') for l in full_table]
        data2 = requests.get(ft_link[0])
        soup2 = BeautifulSoup(data2.text)
        fighter_link = soup2.select('td.b-statistics__table-col')

        for k in fighter_link:
            x = k.find_all('a')
            links = [l.get('href') for l in x]
            l.append(links)
    except:
        continue
    time.sleep(.5)


In [3950]:
# turning the list of lists into a flat list

fl = [item for sublist in l for item in sublist]

In [3951]:
# removes duplicates 

fl = list(set(fl))

In [3958]:
# adds the beginning to the urls

fl_links = [f"http://www.ufcstats.{l.split('.')[1]}" for l in fl]

# Uses the list of links to enter the page of every fighter and extract name and date of birth (dob).

In [4505]:
age_df = pd.DataFrame()
for letter in fl_links:
    try:
        data3 = requests.get(letter)
        soup3 = BeautifulSoup(data3.text)
        bd = soup3.select('li.b-list__box-list-item')[4].text.strip().split(' ')[-3:]
        birthday = bd[0]+' '+bd[1]+' '+bd[2]
        name = soup3.select('span.b-content__title-highlight')[0].text.strip()

        birth_dict = [{'fighter':name, 'dob':birthday}]
        single_df = pd.DataFrame(birth_dict)
        age_df = pd.concat([age_df, single_df])
    except:
        continue
    time.sleep(.5)



# Again, solving the two Bruno Silvas problem

In [4615]:
age_df = age_df.drop(age_df.loc[(age_df.fighter=='Bruno Silva')&(age_df.dob=='Mar 16, 1990')].index)

In [4618]:
age_df.to_csv('age_df', index=False)

# Adding the dob of the fighter and the opponent

In [5308]:
# merging ages onto the figthers

df_f = df_f.merge(age_df, on='fighter', how='left')

In [5309]:
# merging ages onto opponents

df_f = df_f.merge(age_df, left_on='opponent', right_on='fighter')

In [5310]:
df_f = df_f.rename(columns={'fighter_x':'fighter'})

In [5311]:
df_f = df_f.rename(columns={'dob_x':'fighter_dob', 'dob_y':'opp_dob'})

In [5312]:
df_f = df_f.drop(['fighter_y'], axis=1)

In [5317]:
# setting 'dob' column to datetime

df_f['fighter_dob'] = pd.to_datetime(df_f['fighter_dob'], errors='coerce')
df_f['opp_dob'] = pd.to_datetime(df_f['opp_dob'], errors='coerce')

In [5318]:
# calculating age in days

df_f['fighter_age'] = df_f['date'] - df_f['fighter_dob']
df_f['opp_age'] = df_f['date'] - df_f['opp_dob']

In [5319]:
# converting to string to remove day label

df_f[['fighter_age','opp_age']] = df_f[['fighter_age','opp_age']].astype(str)

df_f['fighter_age'] = df_f['fighter_age'].str.replace(' days', '')
df_f['opp_age'] = df_f['opp_age'].str.replace(' days', '')

In [5320]:
# creating a new df

df_f2 = df_f.copy()

# Removing fighters with unlisted ages

In [5326]:
df_f2 = df_f2[(df_f2['fighter_age']!='NaT')&(df_f2['opp_age']!='NaT')]

In [5328]:
# dropping unneed columns

df_f2 = df_f2.drop(['fighter_dob', 'opp_dob', 'ffw', 'ffl'], axis=1)

In [5329]:
# setting age to type int

df_f2['fighter_age'] = df_f2['fighter_age'].astype(int)

df_f2['opp_age'] = df_f2['opp_age'].astype(int)

In [5330]:
# new df

df_f3 = df_f2.copy()

In [5331]:
# treating draws like losses

df_f3['win'] = df_f3['win'].replace(2, 0)

# Creating the final df

In [5333]:
df_f5 = df_f3.copy()

# Changing a loss in the 'win' column from 0 to -1. This is needed to for the function below to work

In [5358]:
df_f5['win2'] = df_f5['win'].where(df_f5['win'] == 1, -1)

df_f5 = df_f5.drop(['win'], axis=1)

df_f5 = df_f5.rename(columns={'win2':'win'})

# Sorting by fighter and date from oldest fight to most recent so I can shift the results down the df and into the future

In [5359]:
df_f5 = df_f5.sort_values(['fighter', 'date'])

# Creating columns for both winning and losing streaks

In [5361]:
def streaks(df):
    s = df['win'].groupby([df['fighter'], (df['win']!=df['win'].shift()).cumsum()]).cumsum()
    return df.assign(w_streak=s.where(s>0, 0), l_streak=s.where(s<1, 0).abs())

df_f5 = streaks(df_f5)

# Shifting streaks ahead one fight date to reflect status before result

In [5362]:
df_f5[['w_streak', 'l_streak']] = df_f5[['w_streak', 'l_streak']].groupby(df_f5['fighter']).shift()

df_f5 = df_f5.fillna(0)

In [5117]:
df_f5.to_csv('df_f5', index=False)

# Merging the df to itself to create the streak columns for the opponent

In [5363]:
df_f5 = df_f5.merge(df_f5, left_on=['opponent', 'date'], right_on=['fighter', 'date'])

In [5111]:
df_f5.columns

Index(['date', 'fighter_x', 'opponent_x', 'fighter_ht_x', 'fighter_reach_x',
       'opponent_ht_x', 'opponent_reach_x', 'wins_x', 'losses_x', 'draw_x',
       'opp_draw_x', 'opp_wins_x', 'opp_losses_x', 'fighter_age_x',
       'opp_age_x', 'win_x', 'w_streak_x', 'l_streak_x', 'fighter_y',
       'opponent_y', 'fighter_ht_y', 'fighter_reach_y', 'opponent_ht_y',
       'opponent_reach_y', 'wins_y', 'losses_y', 'draw_y', 'opp_draw_y',
       'opp_wins_y', 'opp_losses_y', 'fighter_age_y', 'opp_age_y', 'win_y',
       'w_streak_y', 'l_streak_y'],
      dtype='object')

In [5364]:
df_f5 = df_f5.drop(['fighter_y',
       'opponent_y', 'fighter_ht_y', 'fighter_reach_y', 'opponent_ht_y',
       'opponent_reach_y', 'wins_y', 'losses_y', 'draw_y', 'opp_wins_y',
       'opp_losses_y', 'opp_draw_y', 'fighter_age_y', 'opp_age_y', 'win_y'], axis=1)

In [5365]:
df_f5 = df_f5.rename(columns={'fighter_x':'fighter', 'opponent_x':'opponent',\
                              'fighter_ht_x':'fighter_ht', 'fighter_reach_x':'fighter_reach',\
                              'opponent_ht_x':'opp_ht', 'opponent_reach_x':'opp_reach','wins_x':'wins',\
                              'losses_x':'losses', 'draw_x':'draw', 'opp_wins_x':'opp_wins',\
                              'opp_losses_x':'opp_losses', 'opp_draw_x':'opp_draw', 'fighter_age_x':'fighter_age',\
                              'opp_age_x':'opp_age', 'win_x':'win', 'w_streak_x':'w_streak',\
                              'l_streak_x':'l_streak', 'w_streak_y':'opp_w_streak','l_streak_y':'opp_l_streak'})

# Saving the final df that will be used to 

In [5367]:
df_f5.to_csv('df_f5', index=False)

In [5366]:
df_f5.shape

(9836, 20)